# MALTbot Route-B Colab Run-all

사용법:
- Colab에서 **런타임을 GPU**로 설정하세요.
- 좌측 **Secrets**에 `GH_TOKEN`을 추가하세요 (repo push 권한 필요).
- 아래 셀을 위에서부터 그대로 `Run all` 하면 됩니다.
- 이 노트북은 **PR 전용 브랜치 푸시**만 수행하며, `main` 직접 push는 하지 않습니다.


In [ ]:
import torch, platform
print("Python:", platform.python_version())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


In [ ]:
%%bash
set -euo pipefail
REPO_DIR="/content/MALTbot"
REPO_URL="https://github.com/seanwoory/MALTbot.git"

if [ -d "${REPO_DIR}/.git" ]; then
  echo "[info] Repo exists. Pull latest main..."
  cd "${REPO_DIR}"
  git fetch origin
  git checkout main || true
  git pull --ff-only origin main
else
  echo "[info] Cloning repo..."
  git clone "${REPO_URL}" "${REPO_DIR}"
  cd "${REPO_DIR}"
fi

pwd
git rev-parse --short HEAD


In [ ]:
%%bash
set -euo pipefail
python -m pip install -U pip setuptools wheel
python -m pip install \
  "numpy==1.26.4" \
  "scipy==1.11.4" \
  "pandas==2.2.2" \
  "scikit-learn==1.5.2" \
  "pymatgen==2024.8.9" \
  "matbench==0.6" \
  "torch==2.5.1" \
  "pyyaml==6.0.2" \
  "tqdm==4.66.5"


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)


In [ ]:
%%bash
set -euo pipefail
cd /content/MALTbot
python scripts/run_chgnet_mp_e_form.py --config configs/chgnet_mp_e_form.yaml


In [ ]:
%%bash
set -euo pipefail
cd /content/MALTbot
find results -type f -name "results.json" | sort


In [ ]:
from google.colab import userdata
import os

token = userdata.get('GH_TOKEN')
if not token:
    raise ValueError("Colab Secret 'GH_TOKEN'이 비어 있습니다.")
os.environ['GH_TOKEN'] = token
print("GH_TOKEN loaded from Colab Secrets")


In [ ]:
%%bash
set -euo pipefail
cd /content/MALTbot

git config user.name "colab-bot"
git config user.email "colab-bot@users.noreply.github.com"

DATE_KST="$(TZ=Asia/Seoul date +%Y%m%d)"
RUN_TAG="routeb-chgnet-mp-e-form"
BRANCH="bot/${DATE_KST}-${RUN_TAG}"

git checkout -B "${BRANCH}"

if [ -f RESULTS.md ]; then
  git add RESULTS.md
fi
git add results/daily/** || true

if git diff --cached --quiet; then
  echo "[info] Commit할 변경사항이 없습니다."
else
  git commit -m "results: ${RUN_TAG} ${DATE_KST}"

  git -c http.https://github.com/.extraheader="AUTHORIZATION: bearer ${GH_TOKEN}" \
    push -u origin "${BRANCH}"

  echo "[done] Pushed branch: ${BRANCH}"
  echo "PR 생성: https://github.com/seanwoory/MALTbot/compare/main...${BRANCH}?expand=1"
fi
